# 5. Vector Laplacian

Starting off from laplacian formula by taking a scalar function (scalar field) and returning a scalar **p** that measures by how much the average value of f over a surface centered at p deviates from f(p).

![](../images/laplacian.png "Laplacian")

Look at vector Laplacian formula (relevant for Navier stokes viscous stress term) by taking a vector function (vector field) and returning a vector.

![](../images/vector_laplacian.png "Vector Laplacian")

Compute normal component of vector laplacian on finite volume meshes.

![](../images/vector_laplacian_normal_component.png "Normal Component of Vector Laplacian")

Can reuse divergence and curl as defined in previous exercises, however need two more directional gradients:

![](../images/directional_gradient_n_picture.png "Gradient in n direction")

![](../images/directional_gradient_n_formula.png "Gradient in n direction")

![](../images/directional_gradient_tau_picture.png "Gradient in tau direction")

![](../images/directional_gradient_tau_formula.png "Gradient in tau direction")

In [1]:
from helpers import *

import gt4py.next as gtx

In [2]:
def divergence_numpy(
    c2e: np.array,
    u: np.array,
    v: np.array,
    nx: np.array,
    ny: np.array,
    L: np.array,
    A: np.array,
    edge_orientation: np.array,
) -> np.array:
    uv_div = np.sum((u[c2e] * nx[c2e] + v[c2e] * ny[c2e]) * L[c2e] * edge_orientation, axis=1) / A
    return uv_div

In [3]:
def curl_numpy(
    v2e: np.array,
    u: np.array,
    v: np.array,
    nx: np.array,
    ny: np.array,
    dualL: np.array,
    dualA: np.array,
    edge_orientation: np.array,
) -> np.array:
    uv_curl = (
        np.sum(
            (u[v2e] * nx[v2e] + v[v2e] * ny[v2e]) * dualL[v2e] * edge_orientation,
            axis=1,
        )
        / dualA
    )

    return uv_curl

In [4]:
def laplacian_numpy(
    c2e: np.array,
    v2e: np.array,
    e2v: np.array,
    e2c: np.array,
    u: np.array,
    v: np.array,
    nx: np.array,
    ny: np.array,
    L: np.array,
    dualL: np.array,
    tangent_orientation: np.array,
    A: np.array,
    dualA: np.array,
    edge_orientation_vertex: np.array,
    edge_orientation_cell: np.array,
) -> np.array:
    # compute curl (on vertices)
    uv_curl = curl_numpy(v2e, u, v, nx, ny, dualL, dualA, edge_orientation_vertex)

    # compute divergence (on cells)
    uv_div = divergence_numpy(c2e, u, v, nx, ny, L, A, edge_orientation_cell)

    # first term of of nabla2 (gradient of curl)
    grad_of_curl = (uv_curl[e2v[:, 1]] - uv_curl[e2v[:, 0]]) * tangent_orientation / L

    # second term of of nabla2 (gradient of divergence)
    grad_of_div = (uv_div[e2c[:, 1]] - uv_div[e2c[:, 0]]) / dualL

    # finalize nabla2 (difference between the two gradients)
    uv_nabla2 = grad_of_div - grad_of_curl

    return uv_nabla2

In [5]:
@gtx.field_operator
def divergence(
    u: gtx.Field[Dims[E], float],
    v: gtx.Field[Dims[E], float],
    nx: gtx.Field[Dims[E], float],
    ny: gtx.Field[Dims[E], float],
    L: gtx.Field[Dims[E], float],
    A: gtx.Field[Dims[C], float],
    edge_orientation: gtx.Field[Dims[C, C2EDim], float],
) -> gtx.Field[Dims[C], float]:
    # compute divergence
    uv_div = A

    return uv_div

In [6]:
@gtx.field_operator
def curl(
    u: gtx.Field[Dims[E], float],
    v: gtx.Field[Dims[E], float],
    nx: gtx.Field[Dims[E], float],
    ny: gtx.Field[Dims[E], float],
    dualL: gtx.Field[Dims[E], float],
    dualA: gtx.Field[Dims[V], float],
    edge_orientation: gtx.Field[Dims[V, V2EDim], float],
) -> gtx.Field[Dims[V], float]:
    # compute curl
    uv_curl = dualA

    return uv_curl

In [7]:
@gtx.field_operator
def laplacian_fvm(
    u: gtx.Field[Dims[E], float],
    v: gtx.Field[Dims[E], float],
    nx: gtx.Field[Dims[E], float],
    ny: gtx.Field[Dims[E], float],
    L: gtx.Field[Dims[E], float],
    dualL: gtx.Field[Dims[E], float],
    tangent_orientation: gtx.Field[Dims[E], float],
    A: gtx.Field[Dims[C], float],
    dualA: gtx.Field[Dims[V], float],
    edge_orientation_vertex: gtx.Field[Dims[V, V2EDim], float],
    edge_orientation_cell: gtx.Field[Dims[C, C2EDim], float],
) -> gtx.Field[Dims[E], float]:
    # compute laplacian_fvm
    uv_nabla2 = L

    return uv_nabla2

In [ ]:
def test_laplacian():
    backend = None
    # backend = gtfn_cpu
    # backend = gtfn_gpu

    edge_domain = gtx.domain({E: n_edges})
    vertex_domain = gtx.domain({V: n_vertices})
    cell_domain = gtx.domain({C: n_cells})

    u = random_field(edge_domain, allocator=backend)
    v = random_field(edge_domain, allocator=backend)
    nx = random_field(edge_domain, allocator=backend)
    ny = random_field(edge_domain, allocator=backend)
    L = random_field(edge_domain, allocator=backend)
    dualL = random_field(edge_domain, allocator=backend)
    tangent_orientation = random_field(edge_domain, allocator=backend)
    A = random_field(cell_domain, allocator=backend)
    dualA = random_field(vertex_domain, allocator=backend)
    edge_orientation_vertex = random_sign(gtx.domain({V: n_vertices, V2EDim: 6}), allocator=backend)
    edge_orientation_cell = random_sign(gtx.domain({C: n_cells, C2EDim: 3}), allocator=backend)

    laplacian_ref = laplacian_numpy(
        c2e_table,
        v2e_table,
        e2v_table,
        e2c_table,
        u.asnumpy(),
        v.asnumpy(),
        nx.asnumpy(),
        ny.asnumpy(),
        L.asnumpy(),
        dualL.asnumpy(),
        tangent_orientation.asnumpy(),
        A.asnumpy(),
        dualA.asnumpy(),
        edge_orientation_vertex.asnumpy(),
        edge_orientation_cell.asnumpy(),
    )

    c2e_connectivity = gtx.as_connectivity([C, C2EDim], codomain=E, data=c2e_table)
    v2e_connectivity = gtx.as_connectivity([V, V2EDim], codomain=E, data=v2e_table)
    e2v_connectivity = gtx.as_connectivity([E, E2VDim], codomain=V, data=e2v_table)
    e2c_connectivity = gtx.as_connectivity([E, E2CDim], codomain=C, data=e2c_table)

    laplacian_gt4py = gtx.zeros(edge_domain, allocator=backend)

    laplacian_fvm(
        u,
        v,
        nx,
        ny,
        L,
        dualL,
        tangent_orientation,
        A,
        dualA,
        edge_orientation_vertex,
        edge_orientation_cell,
        out=laplacian_gt4py,
        offset_provider={
            C2E.value: c2e_connectivity,
            V2E.value: v2e_connectivity,
            E2V.value: e2v_connectivity,
            E2C.value: e2c_connectivity,
        },
    )

    assert np.allclose(laplacian_gt4py.asnumpy(), laplacian_ref)

In [9]:
test_laplacian()
print("Test successful")

AssertionError: 